# RATIO 2019 - Benchmarking Workshop

In [1]:
import pandas as pd
import numpy as np
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize

# Task 1 - Same Side Clasiification



In [2]:
data_cross_path = 'data/same-side-classification/cross-topic/{}.csv'
data_within_path = 'data/same-side-classification/within-topic/{}.csv'

### Load within-topics and cross-topics data

In [3]:
cross_traindev_df = pd.read_csv(data_cross_path.format('training'), index_col='id')
cross_test_df =  pd.read_csv(data_cross_path.format('test'),index_col='id')

within_traindev_df =  pd.read_csv(data_within_path.format('training'), index_col='id')
within_test_df =  pd.read_csv(data_within_path.format('test'), index_col='id')

In [ ]:
# Adding a tag for the topics in focus: "gay marriage" and "abortion"
def add_tag(row):
    title = row['topic'].lower().strip()
    if title.find('abortion') > -1 :
        row['tag'] = 'abortion'
    elif title.find('gay marriage') > -1 :
        row['tag'] = 'gay marriage'
    else:
        row['tag'] = 'NA'
    return row

cross_traindev_df = cross_traindev_df.apply(add_tag, axis=1)
cross_test_df = cross_test_df.apply(add_tag, axis=1)

within_traindev_df = within_traindev_df.apply(add_tag, axis=1)
within_test_df = within_test_df.apply(add_tag, axis=1)

In [ ]:
cross_traindev_df.head(2)

In [6]:
cross_test_df.head(2)

,argument1,argument1_id,argument2,argument2_id,debate_id,topic,tag
id,,,,,,,
0,i would like to start off by thanking my oppon...,102b48fc-2019-04-18T18:55:38Z-00002-000,i was hoping that since this member took the t...,102b48fc-2019-04-18T18:55:38Z-00002-000,102b48fc-2019-04-18T18:55:38Z,gay marriage is wrong,gay marriage
1,i would like to start off by thanking my oppon...,102b48fc-2019-04-18T18:55:38Z-00002-000,"hello, i am new to this website and usually de...",102b48fc-2019-04-18T18:55:38Z-00002-000,102b48fc-2019-04-18T18:55:38Z,gay marriage is wrong,gay marriage


In [7]:
within_traindev_df.head(2)

,argument1,argument1_id,argument2,argument2_id,debate_id,is_same_side,topic,tag
id,,,,,,,,
85249,"gay marriage devalues marriage, frequency of o...",d2f4b1cd-2019-04-17T11:47:27Z-00063-000,being unaccustomed to gay marriage is no argument,d2f4b1cd-2019-04-17T11:47:27Z-00063-000,d2f4b1cd-2019-04-17T11:47:27Z,False,"gay marriage, debate on same sex marriage",gay marriage
2607,accepted. pro may extend their arguments to th...,2a0d32eb-2019-04-18T11:46:44Z-00004-000,"i""m pro-life. just think about it, your murder...",2a0d32eb-2019-04-18T11:46:44Z-00004-000,2a0d32eb-2019-04-18T11:46:44Z,False,abortion (pro life),abortion


In [8]:
within_test_df.head(2)

,argument1,argument1_id,argument2,argument2_id,debate_id,topic,tag
id,,,,,,,
2,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00001-000,this is my first debate so please just bare wi...,100c174f-2019-04-18T17:33:51Z-00001-000,100c174f-2019-04-18T17:33:51Z,abortion should be illegal with exceptions,abortion
7,first i want to thank my opponent for letting ...,100c174f-2019-04-18T17:33:51Z-00001-000,"in this debate, there are a few factors that m...",100c174f-2019-04-18T17:33:51Z-00001-000,100c174f-2019-04-18T17:33:51Z,abortion should be illegal with exceptions,abortion


## BERTify training and test data

In [11]:
from bert_serving.client import BertClient
bc = BertClient()

In [12]:
a1 = bc.encode(within_traindev_df.argument1.tolist())
a2 = bc.encode(within_traindev_df.argument2.tolist())
traindev_embedded_pairs = zip(a1, a2)
pickle.dump(traindev_embedded_pairs, open("BERT_pairs_train.pkl", "wb"))

/srv/home/gwiedemann/miniconda3/envs/nnnlp/lib/python3.6/site-packages/bert_serving/client/__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=256" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


NameError: name 'test_embedded_pairs' is not defined

In [15]:
a1 = bc.encode(within_test_df.argument1.tolist())
a2 = bc.encode(within_test_df.argument2.tolist())
test_embedded_pairs = zip(a1, a2)
pickle.dump(test_embedded_pairs, open("BERT_pairs_test.pkl", "wb"))

/srv/home/gwiedemann/miniconda3/envs/nnnlp/lib/python3.6/site-packages/bert_serving/client/__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=256" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


## Model

In [ ]:
import keras.backend as K
from keras.models import Sequential, Model, clone_model, load_model
from keras.callbacks import Callback
from keras.optimizers import *
from keras.layers import *
from keras import *
from keras.utils import to_categorical, Sequence
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform

In [29]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [32]:
training_embs = pickle.load(open("BERT_pairs_train.pkl","rb"))

In [33]:
t1, t2 = zip(*training_embs)

In [34]:
args1 = np.array(t1)
args2 = np.array(t2)

In [35]:
args1.shape

(63903, 1024)

In [77]:
output = [1 if t else 0 for t in within_traindev_df.is_same_side]

In [37]:
within_traindev_df.is_same_side[:3]

id
85249    False
2607     False
14632     True
Name: is_same_side, dtype: bool

In [38]:
output

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [101]:
embedding_dims = t1[0].shape

layer_input_1 = Input(shape = embedding_dims, name = 'input_1')
layer_input_2 = Input(shape = embedding_dims, name = 'input_2')

encoder_1 = Dense(embedding_dims[0], activation='relu')(layer_input_1)
encoder_2 = Dense(embedding_dims[0], activation='relu')(layer_input_2)

# combined_diff = subtract([encoder_1, encoder_2])
# combined_mult = multiply([encoder_1, encoder_2])
combined_dot = dot([encoder_1, encoder_2], axes=-1, normalize=False)
combined_all = concatenate([encoder_1, encoder_2, combined_dot])

features_pred = Dense(300, activation='relu')(combined_all)
# features_norm = BatchNormalization()(features_pred)
# features_acti = LeakyReLU()(features_pred)
# features_drop = Dropout(rate=0.5)(features_acti)
layer_prediction = Dense(1, activation='sigmoid')(features_pred)

model = Model([layer_input_1, layer_input_2], layer_prediction)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [102]:
model.fit([args1, args2], output, batch_size=32, epochs=20, validation_split=0.1, verbose=True)

Train on 57512 samples, validate on 6391 samples
Epoch 1/2
57512/57512 [==============================] - 29s 499us/step - loss: 0.3732 - acc: 0.7822 - val_loss: 0.2860 - val_acc: 0.8240
Epoch 2/2
57512/57512 [==============================] - 26s 452us/step - loss: 0.2871 - acc: 0.8269 - val_loss: 0.2748 - val_acc: 0.8284
Train on 57512 samples, validate on 6391 samples
Epoch 3/3
57512/57512 [==============================] - 13s 231us/step - loss: 0.2640 - acc: 0.8405 - val_loss: 0.2676 - val_acc: 0.8352
Train on 57512 samples, validate on 6391 samples
Epoch 4/4
57512/57512 [==============================] - 7s 118us/step - loss: 0.2468 - acc: 0.8519 - val_loss: 0.2645 - val_acc: 0.8376
Train on 57512 samples, validate on 6391 samples
Epoch 4/20
57512/57512 [==============================] - 1s 14us/step - loss: 0.2333 - acc: 0.8644 - val_loss: 0.2670 - val_acc: 0.8363
Epoch 5/20
57512/57512 [==============================] - 1s 14us/step - loss: 0.2280 - acc: 0.8686 - val_loss: 0.26

In [50]:
model.predict([args1, args2])

array([[1.0000000e+00, 1.3444875e-30],
       [9.9147862e-01, 8.5214097e-03],
       [1.5596529e-12, 1.0000000e+00],
       ...,
       [7.6898479e-01, 2.3101521e-01],
       [1.0000000e+00, 3.0680863e-18],
       [4.3383577e-01, 5.6616420e-01]], dtype=float32)

In [ ]:
# dev set performances
# --------------------

# concatenate([encoder_1, encoder_2, combined_diff, combined_dot])
# Epoch 17/20
# 57512/57512 [==============================] - 12s 207us/step - loss: 0.1906 - acc: 0.8963 - val_loss: 0.3081 - val_acc: 0.8512

# combined_all = concatenate([encoder_1, encoder_2])
# Epoch 17/20
# 57512/57512 [==============================] - 11s 191us/step - loss: 0.1767 - acc: 0.9052 - val_loss: 0.3394 - val_acc: 0.8514

# combined_all = concatenate([encoder_1, encoder_2, combined_dot])
# Epoch 17/20
# 57512/57512 [==============================] - 12s 205us/step - loss: 0.1726 - acc: 0.9115 - val_loss: 0.3162 - val_acc: 0.8559

# + dropout 0.1: hülft nicht viel... bzw. nichts...

In [ ]:
# test set performance


report_training_results(y_dev, y_pred)